N-граммы - это языковые модели, которые используются для предсказания N-ного слова исходя из предыдущих N-1 слов.

P(w<sub>i</sub> | w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>) = C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>, w<sub>i</sub>) / C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>)  (1)

Словами: вероятность встретить слово w<sub>i</sub> после слов w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub> (назовём это историей) равна отношению количества раз, которое всё сочетание слов вместе с w<sub>i</sub> втретилось в обучающей выборки, к количеству раз, которое в выборке встретилась история.

Пример: биграммы (N = 2)

In [ ]:
corpus = "John read her book. I read a different book. John read a book by Mulan."

Чтобы P(w<sub>i</sub> | w<sub>i-1</sub>) имело смысл для i = 1, добавим в начало каждого предложения специальный токен &lt;s&gt;. В общем случае таких токенов должно быть N - 1.

Чтобы вероятности всех последовательностей в сумме составляли 1, добавим в конец каждого предложения специальный токен &lt;/s&gt;

Теперь посчитаем вероятность P(John read a book):

P(John read a book) = P(John | &lt;s&gt;) * P(read | John) * P(a | read) * P(book | a) * P(&lt;/s&gt; | book)

А какова вероятность P(Mulan read a book)?

Сглаживание Лапласа: будем делать вид, что все возможные биграммы встретились на один раз больше, чем в реальности. Для этого в формуле (1) прибавим в числителе 1, а в знаменателе - размер словаря (включая &lt;s&gt; и &lt;/s&gt;):

P(w<sub>i</sub> | w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., 
w<sub>i-1</sub>) = (C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>, w<sub>i</sub>) + 1) / (C(w<sub>i-N+1</sub>, w<sub>i-N+2</sub>, ..., w<sub>i-1</sub>) + V) (2)

Обработка неизвестных слов: добавим в словарь токен &lt;unk&gt;, на который будем заменять все незнакомые слова (которые не входят в словарь). Благодаря сглаживанию Лапласа для всех N-грамм с этим словом будут ненулевые вероятности.

Вы также можете оценить вероятность &lt;unk&gt;, заменив на него редкие слова в обучающем корпусе.

Задание: обучить две N-граммные модели на корпусе текстов студенческой конференции (две разные секции или группы секций - например, литературоведение и лингвистика). Взять текст, не входящий в обучающую выборку, и попробовать определить его вероятность по каждой из двух моделей. Где вероятность больше, той секции, скорее всего, он и будет принадлежать.

1. Создать обучающие выборки

1.1. Отобрать тексты для каждой секции

1.2. Каждый текст разбить на предложения, очистить от пунктуации, привести к нижнему регистру

1.3. Каждое предложение разбить на слова, добавить &lt;s&gt; и &lt;/s&gt;, добавить в список предложений

1.4. Должно получиться два списка предложений (по одному на каждую секцию). Каждое предложение - это список слов, первое из которых - &lt;s&gt;, а последнее - &lt;/s&gt;. Следите, чтобы не было пустых предложений.

2. Обучить на каждой выборке N-граммную модель (N = 3 или, в крайнем случае, 2; желательно иметь поддержку произвольного порядка N-грамм)

2.1. Для каждой выборки составить лексикон (т.е. список встретившихся слов)

2.2. В лексикон должны входить &lt;s&gt;, &lt;/s&gt; и &lt;unk&gt;

2.3. Построить два словаря, где ключи - N-граммы и N-1-граммы, значения - их каунты. Эти словари и есть наша модель

3. Использовать модель для определения принадлежности текста

3.1. Взять текст, не входящий ни в одну из выборок (но желательно принадлежащий одной из двух секций)

3.2. Очистить и разбить на предложения по алгоритму из п. 1.

3.3. Для каждого предложения определить вероятность с помощью моделей по формуле (2) (не забывая заменять неизвестные модели слова на &lt;unk&gt; и применять сглаживание Лапласа)

3.4. Определить вероятность текста как произведение вероятностей предложений

3.5. Сравнить полученные вероятности и сделать выводы

4. \* Опциональное задание: написать программу генерации случайных текстов по вашим N-граммам (используя ```random.choices()``` с параметром ```weights```)

Замечание: будет удобнее, если вы будете хранить вероятности не в прямом представлении, а в логарифмическом (основание логарифма может быть любым). Это позволяет:
1. Избежать underflow
2. Заменить умножение на сложение

In [ ]:
!wget https://pkholyavin.github.io/year4programming/conference_stud_2015-2023.pkl

Модуль pickle для сериализации (хранения) данных

In [11]:
import pickle

with open("conference_stud_2015-2023.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
with open("new_pickle.pkl", "wb") as f:
    pickle.dump(data, f)

Посмотрим на список секций

In [12]:
set([i["section"] for i in data])

{'Антрополингвистика: Человек, Язык, Культура',
 'Балканистика. Византинистика. Неоэллинистика',
 'Библеистика',
 'Будетляне. Гипотеза в филологии: научные бои',
 'Грамматика (романо-германская филология)',
 'Грамматика и семантика русского языка',
 'Дискурс и текст',
 'История зарубежных литератур',
 'История и диалектология русского языка',
 'История и теория русского и западноевропейского стиха',
 'История русского языка',
 'Кино|Текст',
 'Киноперевод',
 'Классическая филология',
 'Коллоквиалистика (анализ устной речи)',
 'Компьютерная и прикладная лингвистика',
 'Лексикология (романо-германская филология)',
 'Лексикология и стилистика русского языка',
 'Лингвометодические основы описания и изучения русского языка как иностранного',
 'Литература в междисциплинарной перспективе',
 'Мастерская анализа кинотекста',
 'Междисциплинарная постерная сессия',
 'Общее языкознание',
 'Переводоведение (романо-германская филология)',
 'Пленарное заседание',
 'Прикладная и математическая лингвист